# Get JSON Data for "Simple Sharpe Ratio Optimizer"

In this notebook, we get JSON data for "simple Sharpe ratio optimizer". Make this look academic, model thesis maybe.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import json

## Get data

Let's get the data. Starting with test of 1st 10 tickers

In [2]:
assetTickers = ['AAPL', 'CVX', 'HD', 'JNJ', 'JPM', 'MSFT', 'PG', 'UNH', 'V', 'WMT']

Let's make DataFrame pandas of returns

In [3]:
df = pd.DataFrame()
for ticker in assetTickers:
    # Get monthly all time close data
    tmpDf = pd.DataFrame(yf.Ticker(ticker).history(period="max", interval="1mo")["Close"]).rename(columns={"Close":ticker})
    
    # Format for monthly index using the first datum of each month
    tmpDf.index = tmpDf.index.strftime('%Y-%m')
    tmpDf = tmpDf[~tmpDf.index.duplicated(keep='first')]
    
    df = df.join(tmpDf, how='outer')

# clean data
df.fillna(method='ffill', inplace=True)
df.dropna(how='all', inplace=True)

# get monthly pct return
df = df.pct_change()[1:] * 100

## Get the data we need

Explain each step of how and why I cleaned the data. Explain that I'm doing below to specifically find how each stock directly covaries with each other one.

In [4]:
firstValidMonthDict = dict()
for ticker in assetTickers:
    firstValidMonthDict[ticker] = df[ticker].notna().idxmax()

So now we create the dict

In [5]:
assetDataDict = dict()
for i in range(len(assetTickers)-1):
    assetVar = df[assetTickers[i]].std()**2
    assetDataDict[assetTickers[i]] = {'var': assetVar, 'cov': {}}
    for j in range(i+1, len(assetTickers)):
        startDate = max(firstValidMonthDict[assetTickers[i]], firstValidMonthDict[assetTickers[j]])
        assetCov = df[[assetTickers[i], assetTickers[j]]].loc[startDate:].cov().iloc[0,1]
        assetDataDict[assetTickers[i]]['cov'][assetTickers[j]] = assetCov
assetDataDict[assetTickers[-1]] = {'var': df[assetTickers[-1]].std()**2}

Probably want to discuss why I chose to collect the data in this way fully.

## Export JSON

Export JSON

In [10]:
with open("../simple-sharpe-ratio-optimizer/data/asset-data-test.json", "w") as f:
    json.dump(assetDataDict, f, indent=2) 